In [1]:
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging
import os
from pathlib import Path


DATA_DIR = Path('data') 
KEJADIAN_FILE = DATA_DIR / 'kejadian.csv'


print("--- Skrip Perhitungan Parameter Variogram (Tabel 4.2) ---")


if not KEJADIAN_FILE.exists():
    print(f"❌ ERROR: File {KEJADIAN_FILE} tidak ditemukan.")
    print("Pastikan path direktori dan nama file sudah benar.")
else:
    print(f"✓ File {KEJADIAN_FILE} ditemukan.")
    
    try:
        print("\nLangkah 1: Memuat data...")
        df_crime = pd.read_csv(KEJADIAN_FILE)

        print("Langkah 2: Mempersiapkan data untuk Kriging...")
        lons = df_crime['longitude'].values
        lats = df_crime['latitude'].values
        values = df_crime['jumlah_kejadian'].values
        print(f"✓ {len(values)} titik data siap diproses.")

        print("Langkah 3: Menjalankan OrdinaryKriging (model=spherical)...")
        OK_model = OrdinaryKriging(
            lons, lats, values,
            variogram_model="spherical",
            verbose=False, 
            enable_plotting=False
        )
        print("✓ Model berhasil di-fit.")
        print("Langkah 4: Mengekstrak parameter dan menghitung Sill Total...")
        
        params = OK_model.variogram_model_parameters
        
        sill_parsial = params[0]
        range_val = params[1]
        nugget_val = params[2]
        
        sill_total = sill_parsial + nugget_val

        print("Langkah 5: Membuat tabel hasil...")
        
        data_tabel = {
            'Simbol': ['C', 'a', 'C₀', 'C₀ + C'],
            'Nilai': [sill_parsial, range_val, nugget_val, sill_total]
        }
        
        df_tabel = pd.DataFrame(data_tabel, 
                              index=['Sill Parsial', 'Range', 'Nugget', 'Sill Total'])
        df_tabel.index.name = 'Parameter'
        

        # --- HASIL AKHIR ---
        print("\n" + "="*50)
        print("   HASIL PERHITUNGAN UNTUK TABEL 4.2 (Model Spherical)")
        print("="*50)
        print(df_tabel)
        print("="*50)
        print("\n✓ Selesai. Silakan screenshot atau salin tabel di atas.")

    except Exception as e:
        print(f"❌ Terjadi error saat eksekusi: {e}")

--- Skrip Perhitungan Parameter Variogram (Tabel 4.2) ---
✓ File data\kejadian.csv ditemukan.

Langkah 1: Memuat data...
Langkah 2: Mempersiapkan data untuk Kriging...
✓ 29 titik data siap diproses.
Langkah 3: Menjalankan OrdinaryKriging (model=spherical)...
✓ Model berhasil di-fit.
Langkah 4: Mengekstrak parameter dan menghitung Sill Total...
Langkah 5: Membuat tabel hasil...

   HASIL PERHITUNGAN UNTUK TABEL 4.2 (Model Spherical)
              Simbol     Nilai
Parameter                     
Sill Parsial       C  0.234474
Range              a  0.018354
Nugget            C₀  0.095378
Sill Total    C₀ + C  0.329852

✓ Selesai. Silakan screenshot atau salin tabel di atas.
